In [1]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [2]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A30


In [4]:
# Import dataset from saved PyTorch dataset
data = torch.load("/dfs6/pub/seminl1/graph500/graph500_scale23_ef64.pt")

# Store the dataset to GPU
#data = data.pin_memory()
#data = data.to('cuda:0', non_blocking=True)

In [5]:
# Check whether the dataset is stored on the GPU or not
print(f'Graph is stored on the GPU: {data.is_cuda}')

Graph is stored on the GPU: False


In [6]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[8388608, 64], edge_index=[2, 517557419], y=[8388608])


In [7]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([8388608, 64])
tensor([[-1.6241, -1.1809,  1.4980,  ...,  1.0852, -1.1694,  0.5091],
        [-1.3206, -1.1632, -2.0369,  ..., -2.1283, -1.8054, -1.2949],
        [-1.2896,  1.1596,  1.6661,  ..., -0.4917,  1.7394,  2.0564],
        ...,
        [ 1.0796,  1.1860, -0.0762,  ..., -1.8736,  1.2661,  0.6824],
        [-1.4856,  2.1285, -1.8341,  ..., -1.8353,  0.8483, -0.7442],
        [-1.1195, -0.3819, -0.7515,  ..., -0.5175, -1.4563, -2.4408]])


In [8]:
# Edge index information
print(f'edge_index = {data.edge_index.shape}')
print(data.edge_index)

edge_index = torch.Size([2, 517557419])
tensor([[      0,       0,       0,  ..., 8388607, 8388607, 8388607],
        [      0,       1,       2,  ..., 1320822, 6800756, 8065975]])


In [10]:
# Try to get the number of addition operation
start_point = 0
numAddition = 0
numEdges = 517557419
for i in range(numEdges-1):
    if data.edge_index[0][i+1] == start_point:
        numAddition += 1
    else:
        start_point = data.edge_index[0][i+1]
        if data.edge_index[0][i] % 100000 == 0:
            print(data.edge_index[0][i])
print(numAddition)

tensor(0)
tensor(200000)
tensor(400000)
tensor(600000)
tensor(700000)
tensor(800000)
tensor(900000)
tensor(1000000)
tensor(1100000)
tensor(1200000)
tensor(1300000)
tensor(1400000)
tensor(1600000)
tensor(1700000)
tensor(1800000)
tensor(1900000)
tensor(2000000)
tensor(2100000)
tensor(2200000)
tensor(2300000)
tensor(2400000)
tensor(2500000)
tensor(2600000)
tensor(2700000)
tensor(2800000)
tensor(3000000)
tensor(3100000)
tensor(3200000)
tensor(3400000)
tensor(3500000)
tensor(3600000)
tensor(3700000)
tensor(3800000)
tensor(3900000)
tensor(4000000)
tensor(4100000)
tensor(4200000)
tensor(4300000)
tensor(4500000)
tensor(4600000)
tensor(4800000)
tensor(4900000)
tensor(5000000)
tensor(5300000)
tensor(5400000)
tensor(5500000)
tensor(5600000)
tensor(5700000)
tensor(5900000)
tensor(6100000)
tensor(6200000)
tensor(6300000)
tensor(6400000)
tensor(6500000)
tensor(6700000)
tensor(6800000)
tensor(6900000)
tensor(7000000)
tensor(7200000)
tensor(7300000)
tensor(7500000)
tensor(7700000)
tensor(7800000)
tens

In [11]:
# Ground-truth labels
print(f'y = {data.y.shape}')
print(data.y.dtype)
print(data.y)

y = torch.Size([8388608])
torch.int64
tensor([36, 21, 11,  ...,  6, 46, 53])


In [12]:
from torch_geometric.loader import NeighborLoader

# Create batches with neighbor sampling
train_loader = NeighborLoader(
    data,
    num_neighbors=[30],
    batch_size=2048,
)

In [13]:
# Create a simple GCN with only one GCN layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCNConv(64, 64)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                          lr=0.01,
                                          weight_decay=5e-4)

    def forward(self, x, edge_index):
        x = self.gcn1(x, edge_index)
        z = F.log_softmax(x, dim=1)
        return x, z

In [14]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data, train_loader, device):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 200

    model.train()
    for epoch in range(epochs+1):
        # Training
        total_loss = 0
        acc = 0
        val_loss = 0
        val_acc = 0
        
        # Train on batches
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            h, out = model(batch.x, batch.edge_index)
            loss = criterion(out, batch.y)
            total_loss += loss
            acc += accuracy(out.argmax(dim=1), batch.y)
            loss.backward()
            optimizer.step()

        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {total_loss/len(train_loader):.3f} | Train Acc: '
                  f'{acc/len(train_loader)*100:>6.2f}%')
          
    return model, h, out

In [ ]:
import time

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train and test
# Train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

start_time = time.process_time()
gcn_model, gcn_output, final_output = train(gcn.to(device), data.to(device), train_loader, device)
end_time = time.process_time()
elapsed_time = end_time - start_time
print()
print('Elapsed Time(CPU): ', elapsed_time, 'seconds')

GCN(
  (gcn1): GCNConv(64, 64)
)

Epoch   0 | Train Loss: 4.158 | Train Acc:   2.38%
Epoch  10 | Train Loss: 4.156 | Train Acc:   2.38%
Epoch  20 | Train Loss: 4.157 | Train Acc:   2.38%
Epoch  30 | Train Loss: 4.157 | Train Acc:   2.38%
Epoch  40 | Train Loss: 4.157 | Train Acc:   2.39%
Epoch  50 | Train Loss: 4.157 | Train Acc:   2.39%
Epoch  60 | Train Loss: 4.157 | Train Acc:   2.38%
Epoch  70 | Train Loss: 4.157 | Train Acc:   2.39%
Epoch  80 | Train Loss: 4.157 | Train Acc:   2.39%
Epoch  90 | Train Loss: 4.157 | Train Acc:   2.38%
Epoch 100 | Train Loss: 4.157 | Train Acc:   2.38%
Epoch 110 | Train Loss: 4.157 | Train Acc:   2.39%
Epoch 120 | Train Loss: 4.157 | Train Acc:   2.38%
